<a href="https://colab.research.google.com/github/nAz3626/Distribued-clustering-parking_violation-/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# importing libraries 

import csv
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



# New Section

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 50.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=fad51f1b8892cfe9bdffef9c81b6fb1d70a4b9dfff2acb84bc19147c5e372fed
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("EDA-Parking_violation") \
    .getOrCreate()

In [ ]:
parking = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/Parking_Violations_Issued_-_Fiscal_Year_2017.csv')

In [ ]:
# parking.head(10)
parking.describe().show()

+-------+-------------------+--------+------------------+----------+----------+------------------+------------------+------------+--------------+------------------+------------------+------------------+-----------------------+------------------+------------------+-----------------+------------------+------------------+------------+--------------+-------------------+----------------+---------------------------------+------------------+------------------+-------------------+-------------------+-----------------+------------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+-----------------+------------+-------------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|summary|     Summons Number|Plate ID|Registration State|Plate Type|Issue Date|    Violation Code| Vehicle Body Type|Vehicle Make|Issuing Agency|      Street Code1|      S

In [ ]:
parking.printSchema()


root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

In [ ]:
#for checking Null values
rows=parking.count()
for c in parking.columns:
  
  print(c,parking.where(col(c).isNull()).count()*100/rows)

70460

In [60]:
# print(parking.columns)
pdf=parking.select('*')
# pdf.dropna(how='all').show()

# 
drop_columns=['Summons Number','Hydrant Violation','Double Parking Violation','No Standing or Stopping Violation','Time First Observed','Violation Legal Code','Unregistered Vehicle?','Meter Number']
pdf=pdf.drop(*drop_columns)
pdf.show(10)

+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+----------------+---------------------------------+------------+--------------------+--------------------+-------------------+-----------+------------+--------------------------+--------------------+------------------+-------------+------------+--------------+-------------------+---------------------+
|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation Precinct|Issuer Precinct|Issuer Code|Issuer Command|Issuer Squad|Violation Time|Violation County|Violation In Front Of Or Opposite|House Number|         Street Name| Intersecting Street|Date First Observed|Law

In [59]:
#for checking Null values

for c in pdf.columns:
  
  print(c,pdf.where(col(c).isNull()).count()*100/pdf.count())


Plate ID 0.0
Registration State 0.0
Plate Type 0.0
Issue Date 0.0
Violation Code 0.0
Vehicle Body Type 0.36758444507522
Vehicle Make 0.6741413567981833
Issuing Agency 0.0
Street Code1 0.0
Street Code2 0.0
Street Code3 0.0
Vehicle Expiration Date 0.0
Violation Location 18.508373545273916
Violation Precinct 0.0
Issuer Precinct 0.0
Issuer Code 0.0
Issuer Command 18.391995458416122
Issuer Squad 18.401930173147885
Violation Time 0.0
Time First Observed 92.09906329832529
Violation County 0.33068407607152994
Violation In Front Of Or Opposite 19.30456996877661
House Number 20.509508941243258
Street Name 0.03548112404200965
Intersecting Street 69.5146182231053
Date First Observed 0.001419244961680386
Law Section 0.001419244961680386
Sub Division 0.002838489923360772
Violation Legal Code 81.60800454158388
Days Parking In Effect     24.499006528526824
From Hours In Effect 49.995742265114956
To Hours In Effect 49.995742265114956
Vehicle Color 1.319897814362759
Unregistered Vehicle? 89.787113255747

In [42]:
parking.show(10)

+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+--------------------+--------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation Precinct|Issuer Precinct|I

In [31]:

from pyspark.sql.functions import col,isnan,when,count
parking.select([(count(when(isnan(c) | col(c).isNull(), c))*100/(count(c))).alias(c) for c in parking.columns]
   ).show()

+--------------+--------+------------------+----------+----------+--------------+-------------------+------------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+-----------------+------------------+--------------+-------------------+------------------+---------------------------------+----------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+--------------------+------------------+-----------------+---------------------+--------------------+-----------------+--------------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|  Vehicle Body Type|      Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle E